In [11]:
"""
README: YOLO Detection Failure Visualization Script

This script visualizes detection failures (per-class) by overlaying ground-truth and predicted bounding boxes 
for images listed in a failure analysis JSON (see yolo_failure_analysis.py).
For each class, up to 10 unique examples are visualized:
- original.jpg: original image
- labels.jpg: ground truth boxes overlayed (green)
- detections.jpg: detection results overlayed (class color)

How to use:
- Set the paths for your failure JSON, images, label files, and output directory.
- The script scans all failure entries and saves up to 10 unique visualizations per class.

Requirements:
- Python 3.7+
- OpenCV
- pathlib
- json

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import json
import cv2
from pathlib import Path
from collections import defaultdict

# === Paths ===
failure_json_path = Path("PATH_TO_FAILURE_ANALYSIS_JSON")
images_dir = Path("PATH_TO_IMAGES")
gt_dir = Path("PATH_TO_GROUND_TRUTH_LABELS")
det_dir = Path("PATH_TO_PREDICTION_LABELS")
output_base = Path("failure_visualizations_full")
output_base.mkdir(exist_ok=True, parents=True)

# === Classes and Colors ===
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    0: (0, 255, 255),     # pedestrian - yellow
    1: (255, 0, 255),     # road - magenta
    2: (0, 0, 255),       # vehicle - red
    'gt': (0, 255, 0)     # ground truth - green
}

def draw_boxes(image, boxes, color):
    h, w = image.shape[:2]
    for box in boxes:
        x_c, y_c, bw, bh = box
        x1 = int((x_c - bw / 2) * w)
        y1 = int((y_c - bh / 2) * h)
        x2 = int((x_c + bw / 2) * w)
        y2 = int((y_c + bh / 2) * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
    return image

# Up to 10 unique examples per class
count_per_class = defaultdict(set)

with open(failure_json_path) as f:
    failures = json.load(f)

for entry in failures:
    file_stem = entry.get('file')
    cls_name = entry.get('class')

    if not file_stem or not cls_name:
        continue

    if len(count_per_class[cls_name]) >= 10:
        continue

    if file_stem in count_per_class[cls_name]:
        continue

    cls_id = class_ids.get(cls_name)
    if cls_id is None:
        continue

    image_path = images_dir / f"{file_stem}.jpg"
    gt_path = gt_dir / f"{file_stem}.txt"
    det_path = det_dir / f"{file_stem}.txt"

    if not image_path.exists() or not gt_path.exists():
        continue

    img = cv2.imread(str(image_path))
    if img is None:
        continue

    # Ground truth visualization
    with open(gt_path) as f:
        gt_lines = [line.strip().split() for line in f if line.strip()]
    gt_boxes = [list(map(float, l[1:5])) for l in gt_lines if int(l[0]) == cls_id]

    labels_img = draw_boxes(img.copy(), gt_boxes, colors['gt'])

    # Detection visualization
    det_boxes = []
    if det_path.exists():
        with open(det_path) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
        # Only keep boxes of the target class and confidence >= 0.5
        det_boxes = [list(map(float, l[1:5])) for l in det_lines if int(l[0]) == cls_id and (len(l) < 6 or float(l[5]) >= 0.5)]

    det_img = draw_boxes(img.copy(), det_boxes, colors[cls_id])

    # Save images
    output_folder = output_base / cls_name / file_stem
    output_folder.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_folder / "original.jpg"), img)
    cv2.imwrite(str(output_folder / "labels.jpg"), labels_img)
    cv2.imwrite(str(output_folder / "detections.jpg"), det_img)

    count_per_class[cls_name].add(file_stem)
    print(f"Saved: {output_folder}")


✅ Kaydedildi: failure_visualizations_full/pedestrian/bagdat-1-293_jpg.rf.74450bad7504458a2a6298c5ec855fd1
✅ Kaydedildi: failure_visualizations_full/road/bagdat-1-293_jpg.rf.74450bad7504458a2a6298c5ec855fd1
✅ Kaydedildi: failure_visualizations_full/vehicle/bagdat-1-293_jpg.rf.74450bad7504458a2a6298c5ec855fd1
✅ Kaydedildi: failure_visualizations_full/pedestrian/bagdat-1-4_jpg.rf.665736ea603fff5ddba9e99f9fb4ecf7
✅ Kaydedildi: failure_visualizations_full/vehicle/bagdat-1-4_jpg.rf.665736ea603fff5ddba9e99f9fb4ecf7
✅ Kaydedildi: failure_visualizations_full/pedestrian/muallimnaci-284_jpg.rf.dd3b9eca3aaf5054e147bd82721f6121
✅ Kaydedildi: failure_visualizations_full/vehicle/muallimnaci-284_jpg.rf.dd3b9eca3aaf5054e147bd82721f6121
✅ Kaydedildi: failure_visualizations_full/pedestrian/ciragan-108_jpg.rf.2e2117ac297317c91a05682123abc5a0
✅ Kaydedildi: failure_visualizations_full/vehicle/ciragan-108_jpg.rf.2e2117ac297317c91a05682123abc5a0
✅ Kaydedildi: failure_visualizations_full/pedestrian/katar-1104_